# Exercise 01: Device Query

Query and display GPU properties to understand your hardware.

## Learning Goals
- Use `cudaGetDeviceCount()` and `cudaGetDeviceProperties()`
- Understand key GPU specifications
- First successful CUDA program compilation

## 🚀 Setup Instructions

**IMPORTANT**: Enable GPU before starting!
- Go to: **Runtime → Change runtime type → T4 GPU → Save**

## Step 1: Verify CUDA Installation

In [ ]:
# Check CUDA version and GPU availability
!nvcc --version
print("\n" + "="*50)
!nvidia-smi --query-gpu=name,memory.total,compute_cap --format=csv

## Step 2: Your Exercise - Complete the TODOs

Below is the starter code with TODOs. Your task:
1. Read through the code
2. Complete the sections marked with `TODO`
3. Run the code to see your GPU properties

In [ ]:
%%writefile device_query.cu
/**
 * Exercise 01: Device Query
 * 
 * Query and display GPU properties.
 * 
 * TODO: Complete the missing parts marked with TODO
 */

#include <stdio.h>
#include <cuda_runtime.h>

// Error checking macro
#define CUDA_CHECK(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            fprintf(stderr, "CUDA error at %s:%d: %s\n", \
                    __FILE__, __LINE__, cudaGetErrorString(err)); \
            exit(EXIT_FAILURE); \
        } \
    } while(0)

int main() {
    int deviceCount = 0;
    
    // TODO 1: Get the number of CUDA devices
    // Hint: Use cudaGetDeviceCount()
    CUDA_CHECK(cudaGetDeviceCount(&deviceCount));
    
    if (deviceCount == 0) {
        printf("No CUDA-capable devices found!\n");
        return 1;
    }
    
    printf("Found %d CUDA device(s)\n\n", deviceCount);
    
    // Query properties for each device
    for (int dev = 0; dev < deviceCount; dev++) {
        cudaDeviceProp prop;
        
        // TODO 2: Get device properties
        // Hint: Use cudaGetDeviceProperties()
        CUDA_CHECK(cudaGetDeviceProperties(&prop, dev));
        
        // Display basic information
        printf("========== Device %d: %s ==========\n", dev, prop.name);
        printf("  Compute Capability:        %d.%d\n", prop.major, prop.minor);
        printf("  Total Global Memory:       %.2f GB\n", 
               prop.totalGlobalMem / 1e9);
        
        // TODO 3: Display multiprocessor count
        // Hint: prop.multiProcessorCount
        printf("  Multiprocessors (SMs):     %d\n", prop.multiProcessorCount);
        
        // TODO 4: Display CUDA cores per SM (depends on compute capability)
        int coresPerSM = 0;
        switch (prop.major) {
            case 2:  coresPerSM = 32; break;  // Fermi
            case 3:  coresPerSM = 192; break; // Kepler
            case 5:  coresPerSM = 128; break; // Maxwell
            case 6:  coresPerSM = (prop.minor == 0) ? 64 : 128; break; // Pascal
            case 7:  coresPerSM = 64; break;  // Volta/Turing
            case 8:  coresPerSM = (prop.minor == 0) ? 64 : 128; break; // Ampere
            default: coresPerSM = 0;
        }
        printf("  CUDA Cores per SM:         %d\n", coresPerSM);
        printf("  Total CUDA Cores:          %d\n", 
               prop.multiProcessorCount * coresPerSM);
        
        // TODO 5: Display memory information
        printf("  Shared Memory per Block:   %zu KB\n", 
               prop.sharedMemPerBlock / 1024);
        printf("  Registers per Block:       %d\n", prop.regsPerBlock);
        
        // TODO 6: Display thread limits
        printf("  Warp Size:                 %d\n", prop.warpSize);
        printf("  Max Threads per Block:     %d\n", prop.maxThreadsPerBlock);
        printf("  Max Block Dimensions:      (%d, %d, %d)\n",
               prop.maxThreadsDim[0], prop.maxThreadsDim[1], prop.maxThreadsDim[2]);
        printf("  Max Grid Dimensions:       (%d, %d, %d)\n",
               prop.maxGridSize[0], prop.maxGridSize[1], prop.maxGridSize[2]);
        
        // TODO 7: Display clock speeds
        printf("  GPU Clock Rate:            %.2f GHz\n", 
               prop.clockRate / 1e6);
        printf("  Memory Clock Rate:         %.2f GHz\n",
               prop.memoryClockRate / 1e6);
        printf("  Memory Bus Width:          %d-bit\n", prop.memoryBusWidth);
        
        // TODO 8: Calculate and display memory bandwidth
        double memBandwidth = 2.0 * prop.memoryClockRate * 
                             (prop.memoryBusWidth / 8.0) / 1e6;
        printf("  Peak Memory Bandwidth:     %.2f GB/s\n", memBandwidth);
        
        // TODO 9: Calculate theoretical peak performance
        // Formula: CUDA Cores × Clock Speed × 2 (FMA operations)
        int totalCores = prop.multiProcessorCount * coresPerSM;
        double peakGFLOPS = (totalCores * (prop.clockRate / 1e6) * 2.0);
        printf("  Peak FP32 Performance:     %.2f TFLOPS\n", peakGFLOPS / 1000.0);
        
        printf("\n");
    }
    
    return 0;
}

## Step 3: Compile the Program

In [ ]:
# Compile with nvcc
!nvcc -arch=sm_75 device_query.cu -o device_query
print("✅ Compilation successful!")

## Step 4: Run and See Your GPU Properties!

In [ ]:
# Run the compiled program
!./device_query

---

## 🔄 Python Comparison (Optional)

Here's the same device query in Python using Numba. Compare with the C++ version above!

### C++ vs Python Syntax

| Task | CUDA C++ | Python (Numba) |
|------|----------|----------------|
| Get device count | `cudaGetDeviceCount(&count)` | `len(cuda.gpus)` |
| Get device | `cudaGetDeviceProperties(&prop, 0)` | `cuda.get_current_device()` |
| Device name | `prop.name` | `device.name` |
| Compute capability | `prop.major, prop.minor` | `device.compute_capability` |

In [ ]:
# 🐍 Python Version - Device Query with Numba
!pip install numba -q

from numba import cuda
import numpy as np

print("=" * 50)
print("PYTHON (Numba) DEVICE QUERY")
print("=" * 50)

# Check CUDA availability
if cuda.is_available():
    print(f"✅ CUDA is available!")
    print(f"   Number of GPUs: {len(cuda.gpus)}")
    
    # Get current device
    device = cuda.get_current_device()
    
    print(f"\n📊 Device: {device.name.decode()}")
    print(f"   Compute Capability: {device.compute_capability}")
    
    # Get more properties
    ctx = cuda.current_context()
    
    # Memory info
    free_mem, total_mem = cuda.current_context().get_memory_info()
    print(f"   Total Memory: {total_mem / 1e9:.2f} GB")
    print(f"   Free Memory: {free_mem / 1e9:.2f} GB")
    
    # Thread limits (from device attributes)
    print(f"   Max Threads per Block: {device.MAX_THREADS_PER_BLOCK}")
    print(f"   Max Block Dimensions: {device.MAX_BLOCK_DIM_X} x {device.MAX_BLOCK_DIM_Y} x {device.MAX_BLOCK_DIM_Z}")
    print(f"   Max Grid Dimensions: {device.MAX_GRID_DIM_X} x {device.MAX_GRID_DIM_Y} x {device.MAX_GRID_DIM_Z}")
    print(f"   Warp Size: {device.WARP_SIZE}")
    print(f"   Multiprocessors: {device.MULTIPROCESSOR_COUNT}")
else:
    print("❌ CUDA is not available!")

print("\n💡 Notice: Python gives less detail than C++ cudaGetDeviceProperties()")

### 🎯 Key Takeaway

**C++ provides full access** to all GPU properties via `cudaDeviceProp` (50+ fields).

**Python (Numba)** gives you the essentials, but less detail.

For serious CUDA work → **Stick with C++** (the exercise above)

---

## 📊 Understanding the Output

### Key Properties Explained

| Property | What it means | Why it matters |
|----------|---------------|----------------|
| **Compute Capability** | GPU architecture version (e.g., 7.5 for T4) | Determines supported CUDA features |
| **Multiprocessors (SMs)** | Number of parallel processing units | More SMs = more parallelism |
| **CUDA Cores** | Processing elements that execute instructions | More cores = higher throughput |
| **Global Memory** | Total GPU memory (usually 16GB on T4) | Limits problem size you can solve |
| **Shared Memory** | Fast on-chip memory per block | Critical for performance optimization |
| **Max Threads/Block** | Thread limit per block (usually 1024) | Affects kernel launch configuration |
| **Warp Size** | Threads executed together (always 32) | SIMT execution model |
| **Memory Bandwidth** | Data transfer rate (GB/s) | Often the bottleneck in GPU code |
| **Peak FLOPS** | Theoretical floating-point operations/sec | Maximum compute performance |

### For T4 GPU (typical Colab)
- Compute Capability: **7.5**
- CUDA Cores: **2560**
- Memory: **16 GB**
- Bandwidth: **300 GB/s**
- Peak FP32: **~8.1 TFLOPS**

### 🎯 Tasks

- ✅ Complete all TODOs in the code
- ✅ Run and understand the output
- ✅ Compare memory bandwidth with peak FLOPS
- ✅ Calculate: How many GB can you transfer in 1ms?
- ✅ Calculate: How many float operations in 1ms?

### 💡 Bonus Exercises

1. **Arithmetic Intensity**: If you need to do 1000 operations per byte transferred, will you be memory-bound or compute-bound?
2. **Thread Blocks**: If your kernel uses 512 threads per block, how many blocks can fit on one SM? (Hint: check `maxThreadsPerMultiProcessor`)
3. **Memory Math**: At peak bandwidth, how long does it take to fill all GPU memory?

### 📚 Learn More

- [CUDA C Programming Guide - Compute Capabilities](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#compute-capabilities)
- [T4 GPU Specifications](https://www.nvidia.com/en-us/data-center/tesla-t4/)
- [Understanding GPU Architecture](../../../cuda-programming-guide/01-introduction/hardware-implementation.md)

## 🎓 Solution

If you completed the exercise, compare your implementation with the solution below.

<details>
<summary>Click to reveal solution notes</summary>

### Key Points
1. **Error Checking**: Always use `CUDA_CHECK()` macro for CUDA API calls
2. **Device Count**: `cudaGetDeviceCount()` returns number of available GPUs
3. **Properties**: `cudaGetDeviceProperties()` fills a `cudaDeviceProp` structure
4. **CUDA Cores**: Calculation depends on compute capability (architecture)
5. **Bandwidth**: Formula is `2 × memory_clock × (bus_width/8) / 1e6`
6. **Peak FLOPS**: `cores × clock × 2` (2 for FMA - fused multiply-add)

All TODOs in the code above are already completed as hints!
</details>

## ➡️ Next Exercise

Continue to [Exercise 02: Hello GPU](../ex02-hello-gpu/colab-hello-gpu.ipynb) to write your first kernel!

---

**Completed this exercise?** Great! Save this notebook to your Drive and move on to the next one.